In [1]:
import uproot
import numpy as np
import pandas as pd

index       = [ "B_sig_K_dr","B_sig_K_dz","B_sig_CleoConeCS_3_ROE", "thrustAxisCosTheta","aplanarity","sphericity",
                "harmonicMomentThrust0","harmonicMomentThrust1","harmonicMomentThrust2","harmonicMomentThrust3","harmonicMomentThrust4",
                "foxWolframR1","foxWolframR2","foxWolframR3","foxWolframR4", "B_sig_isSignalAcceptMissingNeutrino"]

data        = uproot.open("/ceph/aavocone/Data/forAlex/processed_simulation_B_K_a_nunu_ma_4_6_GeV_100000_events_nobdtcut.root:tree_Bsig;1").arrays(index, library ="pd")
charged     = uproot.open('/ceph/aavocone/Data/forAlex/kplus_v34_kshort_v34_100invfb_test_nobdtcut_16812.root:tree_Bsig;1').arrays(index, library ="pd")
mixed       = uproot.open('/ceph/aavocone/Data/forAlex/kplus_v34_kshort_v34_100invfb_test_nobdtcut_16817.root:tree_Bsig;1').arrays(index, library ="pd")
uu          = uproot.open('/ceph/aavocone/Data/forAlex/kplus_v34_kshort_v34_100invfb_test_nobdtcut_16802.root:tree_Bsig;1').arrays(index, library ="pd")
cc          = uproot.open('/ceph/aavocone/Data/forAlex/kplus_v34_kshort_v34_100invfb_test_nobdtcut_16792.root:tree_Bsig;1').arrays(index, library ="pd")
dd          = uproot.open('/ceph/aavocone/Data/forAlex/kplus_v34_kshort_v34_100invfb_test_nobdtcut_16797.root:tree_Bsig;1').arrays(index, library ="pd")
ss          = uproot.open('/ceph/aavocone/Data/forAlex/kplus_v34_kshort_v34_100invfb_test_nobdtcut_16807.root:tree_Bsig;1').arrays(index, library ="pd")

sets =[data,charged,mixed,uu,cc,dd,ss]
l=0
for i,v in enumerate(sets):
    if i==0:
        v["signal"]  = np.ones(len(v))
        
    else:
        v["signal"] = np.zeros(len(v))
    v["class"] = np.ones(len(v))*i
    l += len(v)
data.drop(data[data["B_sig_isSignalAcceptMissingNeutrino"]==0.0].index, inplace = True)
df = pd.concat(sets)
df.to_csv("/work/aavocone/4_6_set.csv", index= False)


In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

#test train split
X = df[df.columns[:-3]]    #exclude "signal" "classification" "B_sig_isSignalAcceptMissingNeutrino"
y = df["signal"]            
xtrain,xtest,ytrain,ytest = train_test_split(X, y, test_size = 0.33, stratify = y)
xtrain,xval,ytrain,yval = train_test_split(xtrain, ytrain, test_size = 0.5)



weight = (len(ytest)-sum(ytest))/sum(ytest)

#train with scale_pos_weights
model     = XGBClassifier(n_estimators = 500, learning_rate = 0.2, eval_metric = "logloss", scale_pos_weight = weight, use_label_encoder =False)
model.fit(xtrain,ytrain,eval_set=[xval,yval],verbose =True)
model.save_model("model_4_6.txt")


,B_sig_K_dr,B_sig_K_dz,B_sig_CleoConeCS_3_ROE,thrustAxisCosTheta,aplanarity,sphericity,harmonicMomentThrust0,harmonicMomentThrust1,harmonicMomentThrust2,harmonicMomentThrust3,harmonicMomentThrust4,foxWolframR1,foxWolframR2,foxWolframR3,foxWolframR4,B_sig_isSignalAcceptMissingNeutrino,signal,class
0,0.005391,0.016027,0.097269,0.359121,0.019786,0.179735,0.437424,0.095752,0.254479,0.067192,0.123666,0.142815,0.370648,0.091472,0.310429,1.0,1.0,0.0
1,0.004240,0.018286,0.000000,0.255490,0.019037,0.082969,0.480654,0.172493,0.353945,0.119777,0.245487,0.151586,0.547385,0.117421,0.361651,1.0,1.0,0.0
3,0.002521,-0.044096,0.418772,0.583204,0.098089,0.486478,0.684936,-0.154365,0.243203,-0.153694,0.056883,0.069033,0.157963,0.295088,0.113430,1.0,1.0,0.0
4,0.001137,0.017522,0.000000,0.674745,0.057328,0.175479,0.577686,0.019343,0.392771,0.032639,0.201706,0.020092,0.471712,0.102639,0.204832,1.0,1.0,0.0
5,0.001230,0.051630,0.646702,0.089957,0.086164,0.280212,0.604341,-0.204965,0.317579,-0.100185,0.165141,0.115173,0.294946,0.064938,0.144696,1.0,1.0,0.0


In [ ]:
print(weight)